In [1]:
clientId = "9e443dbb-73f2-4e20-bf4d-fffcd86cd926"
clientKey = "1]1mmsSjBHe0PXzD8trcJsxWh.iOUa_/"
tenantId = "fd89f191-166e-474f-a80f-af83c8f5e17e"
configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": clientId,
           "fs.azure.account.oauth2.client.secret": clientKey,
           "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/"+tenantId+"/oauth2/token"}

# # Optionally, you can add <directory-name> to the source URI of your mount point.
dbutils.fs.mount(
 source = "abfss://rawdata@team9adls.dfs.core.windows.net/",
 mount_point = "/mnt/rawdata",
 extra_configs = configs)

In [2]:
import pandas as pd

Sales_Addresses_FP = "/mnt/rawdata/cloudsales/dboAddresses.csv"
Streaming_Addresses_FP = "/mnt/rawdata/cloudstreaming/dboAddresses.csv"

Sales_Addresses_Raw = sqlContext.read.parquet(Sales_Addresses_FP)
Streaming_Addresses_Raw = sqlContext.read.parquet(Streaming_Addresses_FP)

Sales_Addresses_Raw = Sales_Addresses_Raw.toPandas()
Streaming_Addresses_Raw = Streaming_Addresses_Raw.toPandas()

Sales_Addresses_DF = [Sales_Addresses_Raw, Streaming_Addresses_Raw]
Cloud_Addresses = pd.concat(Sales_Addresses_DF)

Cloud_Addresses['CreatedDate'] = pd.to_datetime(Cloud_Addresses['CreatedDate'], errors='coerce')
Cloud_Addresses['UpdatedDate'] = pd.to_datetime(Cloud_Addresses['UpdatedDate'], errors='coerce')

Cloud_Addresses.AddressLine2 = Cloud_Addresses.AddressLine2.astype(str)
Cloud_Addresses.ZipCode = Cloud_Addresses.ZipCode.astype(str)

display(Cloud_Addresses)

In [3]:
VanCustomers_FP = "/mnt/rawdata/vanarsdell/dboCustomers.txt"

VanCustomers_raw = spark.read.json(VanCustomers_FP)
VanCustomers_df = VanCustomers_raw.toPandas()

VanAddresses = VanCustomers_df[['CustomerID', 'AddressLine1', 'AddressLine2', 'City', 'State', 'ZipCode', 'CreatedDate', 'UpdatedDate']]

VanAddresses['CreatedDate'] = pd.to_datetime(VanAddresses['CreatedDate'], errors='coerce')
VanAddresses['UpdatedDate'] = pd.to_datetime(VanAddresses['UpdatedDate'], errors='coerce')

VanAddresses.AddressLine2 = VanAddresses.AddressLine2.astype(str)
VanAddresses.ZipCode = VanAddresses.ZipCode.astype(str)

VanAddresses.insert(0, 'AddressID', 'None')

display(VanAddresses)

In [4]:
CoffeeCustomers_FP = "/mnt/rawdata/fourthcoffee/Customers.csv"

CoffeeCustomer_raw = spark.read.csv(CoffeeCustomers_FP, header=True)

CoffeeCustomers_df = CoffeeCustomer_raw.toPandas()

CoffeeAddresses = CoffeeCustomers_df[['CustomerID', 'AddressLine1', 'AddressLine2', 'City', 'State', 'ZipCode', 'CreatedDate', 'UpdatedDate']]

CoffeeAddresses['CreatedDate'] = pd.to_datetime(CoffeeAddresses['CreatedDate'], errors='coerce')
CoffeeAddresses['UpdatedDate'] = pd.to_datetime(CoffeeAddresses['UpdatedDate'], errors='coerce')
CoffeeAddresses.AddressLine2 = CoffeeAddresses.AddressLine2.astype(str)
CoffeeAddresses.ZipCode = CoffeeAddresses.ZipCode.astype(str)

CoffeeAddresses.loc[CoffeeAddresses.AddressLine2 == 'nan', 'AddressLine2'] = 'None'

CoffeeAddresses.insert(0, 'AddressID', 'None')

display(CoffeeAddresses)

In [5]:
Cloud_Addresses['SourceSystem'] = '1'
VanAddresses['SourceSystem'] = '2'
CoffeeAddresses['SourceSystem'] = '3'
addresses_frame = [Cloud_Addresses, VanAddresses, CoffeeAddresses]

all_addresses = pd.concat(addresses_frame)

display(all_addresses)

In [6]:
all_addresses = spark.createDataFrame(all_addresses)
all_addresses.coalesce(1).write.mode("overwrite").format("com.databricks.spark.csv").option("header","true").csv("/mnt/rawdata/stagingdata/AllAddresses/")